In [31]:


import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv
import datetime

warnings.filterwarnings("ignore")

In [32]:
load_dotenv()
os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2024-12-21 14:33:26,429 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-21 14:33:26,430 INFO: Initializing external client
2024-12-21 14:33:26,431 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-21 14:33:28,755 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


In [33]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

today = datetime.date.today()

today


datetime.date(2024, 12, 21)

In [34]:

# Step 2: Retrieve the feature group by name and version
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=6
)

# Step 3: Read the data from the feature group
s_data_df = solana_feature_group.select(["date"]).read()

# Step 4: Find the maximum date
solana_max_date = s_data_df['date'].max()

# Step 5: Print the result
print(f"The maximum date is: {solana_max_date}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 
The maximum date is: 2024-12-21 00:00:00+00:00


In [35]:

# Step 2: Retrieve the feature group by name and version
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=6
)

# Step 3: Read the data from the feature group
b_data_df = bitcoin_feature_group.select(["date"]).read()

# Step 4: Find the maximum date
bitcoin_max_date = b_data_df['date'].max()

# Step 5: Print the result
print(f"The maximum date is: {bitcoin_max_date}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.82s) 
The maximum date is: 2024-12-21 00:00:00+00:00


In [37]:
# Step 2: Retrieve the feature group by name and version
fng_feature_group = fs.get_feature_group(
    name="f_n_g_index", 
    version=6
)

# Step 3: Read the data from the feature group
fng_data_df = fng_feature_group.select(["date"]).read()

# Step 4: Find the maximum date
fng_max_date = fng_data_df['date'].max()

# Step 5: Print the result
print(f"The maximum date is: {fng_max_date}")

2024-12-21 14:33:50,968 ERROR: [Errno 2] Opening HDFS file '/apps/hive/warehouse/kth_lab1_2024_featurestore.db/f_n_g_index_6/.hoodie/hoodie.properties' failed. Detail: [errno 2] No such file or directory. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {grpc_message:"[Errno 2] Opening HDFS file \'/apps/hive/warehouse/kth_lab1_2024_featurestore.db/f_n_g_index_6/.hoodie/hoodie.properties\' failed. Detail: [errno 2] No such file or directory. Detail: Python exception: FlyingDuckException", grpc_status:2, created_time:"2024-12-21T14:33:50.967868+02:00"}. Client context: IOError: Server never sent a data message. Detail: Internal
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/final_project_env/lib/python3.10/site-packages/hsfs/core/arrow_flight_client.py", line 364, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
  File "/opt/homebrew/anaconda3/envs/final_project_env

FeatureStoreException: Could not read data using Hopsworks Feature Query Service.

### Getting missing data

### Request solana price from last available data till now

In [ ]:
# convert the date to a naive date
solana_max_date_naive = solana_max_date.date()

# Calculate the difference in days, so how many days ago was the last data point
solana_difference = (solana_max_date_naive - today).days
print(f"Difference in days: {solana_difference}")

In [ ]:
url = "https://min-api.cryptocompare.com/data/v2/histoday"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


# Set the parameters for the API request
params = {
    "fsym": "SOL",  # From symbol (e.g., Bitcoin)
    "tsym": "USD",  # To symbol (e.g., US Dollar)
    "limit": solana_difference,    # Number of days of data to retrieve
    "toTs": None,   # Timestamp for the end of the period (None for latest data)
    "api_key": crypto_api_key
}

sol_response = trigger_request(url, params)
sol_response.keys()

In [ ]:
sol_response["Data"]

In [ ]:
# convert the date to a naive date
bitcoin_max_date_naive = bitcoin_max_date.date()

# Calculate the difference in days, so how many days ago was the last data point
bitcoin_difference = (bitcoin_max_date_naive - today).days
print(f"Difference in days: {bitcoin_difference}")

In [ ]:
url = "https://min-api.cryptocompare.com/data/v2/histoday"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


# Set the parameters for the API request
params = {
    "fsym": "BTC",  # From symbol (e.g., Bitcoin)
    "tsym": "USD",  # To symbol (e.g., US Dollar)
    "limit": bitcoin_difference,    # Number of days of data to retrieve
    "toTs": None,   # Timestamp for the end of the period (None for latest data)
    "api_key": crypto_api_key
}

btc_response = trigger_request(url, params)
btc_response.keys()

In [ ]:
response["Data"]

In [ ]:

# convert the date to a naive date
fng_max_date_naive = fng_max_date.date()

# Calculate the difference in days, so how many days ago was the last data point
fng_difference = (fng_max_date_naive - today).days
print(f"Difference in days: {fng_difference}")

In [27]:
url = "https://api.alternative.me/fng/?"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


# Set the parameters for the API request
params = {
    "limit": fng_difference    # Number of days of data to retrieve
}

response = trigger_request(url, params)
response.keys()

dict_keys(['name', 'data', 'metadata'])

In [30]:
response['data']

[{'value': '73',
  'value_classification': 'Greed',
  'timestamp': '1734739200',
  'time_until_update': '41404'},
 {'value': '74', 'value_classification': 'Greed', 'timestamp': '1734652800'},
 {'value': '75', 'value_classification': 'Greed', 'timestamp': '1734566400'},
 {'value': '81',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734480000'},
 {'value': '87',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734393600'},
 {'value': '83',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734307200'},
 {'value': '80',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734220800'},
 {'value': '83',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734134400'},
 {'value': '76',
  'value_classification': 'Extreme Greed',
  'timestamp': '1734048000'},
 {'value': '83',
  'value_classification': 'Extreme Greed',
  'timestamp': '1733961600'},
 {'value': '74', 'value_classification': 'Greed', 'timestamp': '1733875200'},
 {'value': '78',
  'va